<a href="https://colab.research.google.com/github/helldoodle-dev/voice-ai/blob/main/Changer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Real-Time Voice Changer
##### 😉
---
Colab by helldoodle based on the [voice-ai](https://github.com/helldoodle)

In [ ]:
#@title  { display-mode: "form" }
#@markdown ## <- Запуск, но перед этим:

import os
import subprocess
import sys
from tqdm import tqdm
import requests

repo_dir = '/content/voice-ai'

# Проверка наличия репозитория
if os.path.exists(repo_dir):
    print('Код выполняется повторно. Если вы изменили свой ник или список стримеров, во избежание ошибок, пожалуйста, перезапустите среду выполнения: \nСреда выполнения -> Отключиться от среды выполнения и удалить её -> Да\n')

else:
    print("\nЗагрузка..")
    !git clone https://github.com/helldoodle-dev/voice-ai.git --quiet

    def install(package):
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

    with open(f'{repo_dir}/requirements.txt', 'r') as file:
        packages = file.read().splitlines()

    for package in tqdm(packages):
        install(package)

    !apt-get -y install libportaudio2 -qq
    print("\033[32mСкачивание завершено!\033[0m")
    print("\nЗапуск программы..")

#@markdown Введите **Ваш** токен [Ngrok](https://dashboard.ngrok.com/get-started/your-authtoken):
token = "your-token" #@param {type:"string"}

#@markdown ...

#@markdown Выберите ближайший сервер к вам:
region = "us - United States (Ohio)" # @param ["ap - Asia/Pacific (Singapore)", "au - Australia (Sydney)","eu - Europe (Frankfurt)", "in - India (Mumbai)","jp - Japan (Tokyo)","sa - South America (Sao Paulo)", "us - United States (Ohio)"]

from pyngrok import conf, ngrok
MyConfig = conf.PyngrokConfig()
MyConfig.auth_token = token
MyConfig.region = region[0:2]
#conf.get_default().authtoken = token
#conf.get_default().region = region
conf.set_default(MyConfig);

import subprocess, threading, time, socket, urllib.request
PORT = 8000

from pyngrok import ngrok
ngrokConnection = ngrok.connect(PORT)
public_url = ngrokConnection.public_url

from IPython.display import clear_output

def wait_for_server():
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', PORT))
        if result == 0:
            break
        sock.close()

    clear_output()
    print("--------- SERVER READY! ---------")
    print("Your server is available at:")
    print(public_url)
    print("---------------------------------")

threading.Thread(target=wait_for_server, daemon=True).start()

%cd /content/voice-ai/
!python3 MMVCServerSIO.py \
  -p {PORT} \
  --https False \
  --content_vec_500 pretrain/checkpoint_best_legacy_500.pt \
  --content_vec_500_onnx pretrain/content_vec_500.onnx \
  --content_vec_500_onnx_on true \
  --hubert_base pretrain/hubert_base.pt \
  --hubert_base_jp pretrain/rinna_hubert_base_jp.pt \
  --hubert_soft pretrain/hubert/hubert-soft-0d54a1f4.pt \
  --nsf_hifigan pretrain/nsf_hifigan/model \
  --crepe_onnx_full pretrain/crepe_onnx_full.onnx \
  --crepe_onnx_tiny pretrain/crepe_onnx_tiny.onnx \
  --rmvpe pretrain/rmvpe.pt \
  --model_dir model_dir \
  --samples samples.json

ngrok.disconnect(ngrokConnection.public_url)